## Importing Libraries and Data

In [4]:
!pip install pandas
!pip install tensorflow
!pip install tensorflow_hub
!pip install tokenization
!pip install bert-tensorflow==1.0.1

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import re
from bert import tokenization
import string
tf.gfile = tf.io.gfile

import os

os.getcwd()

'/content'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

# 구글드라이브 경로
train = pd.read_csv("/content/drive/MyDrive/python/train.csv")
test = pd.read_csv("/content/drive/MyDrive/python/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/python/sample_submission.csv")

Mounted at /content/drive


## Helper functions

In [7]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:

        text = tokenizer.tokenize(text)
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len

        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [8]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Data cleaning

In [9]:
def lowercase_text(text):
    return text.lower()

train.text=train.text.apply(lambda x: lowercase_text(x))
test.text=test.text.apply(lambda x: lowercase_text(x))

In [10]:
def remove_noise(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

train.text=train.text.apply(lambda x: remove_noise(x))
test.text=test.text.apply(lambda x: remove_noise(x))

In [11]:
train.text.head(5)

0    our deeds are the reason of this earthquake ma...
1                forest fire near la ronge sask canada
2    all residents asked to shelter in place are be...
3     people receive wildfires evacuation orders in...
4    just got sent this photo from ruby alaska as s...
Name: text, dtype: object

## Pre-Training (BERT)

In [12]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [13]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [14]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

## Modelling

In [15]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=8,
    callbacks=[checkpoint],
    batch_size=10
)

Epoch 1/8
609/609 [==============================] - 1293s 2s/step - loss: 0.4292 - accuracy: 0.8126 - val_loss: 0.3748 - val_accuracy: 0.8431
Epoch 2/8
609/609 [==============================] - 1253s 2s/step - loss: 0.2535 - accuracy: 0.9013 - val_loss: 0.3976 - val_accuracy: 0.8299
Epoch 3/8
609/609 [==============================] - 1252s 2s/step - loss: 0.1126 - accuracy: 0.9586 - val_loss: 0.5111 - val_accuracy: 0.8286
Epoch 4/8
609/609 [==============================] - 1252s 2s/step - loss: 0.0765 - accuracy: 0.9713 - val_loss: 0.7339 - val_accuracy: 0.8175
Epoch 5/8
609/609 [==============================] - 1252s 2s/step - loss: 0.0612 - accuracy: 0.9741 - val_loss: 0.6615 - val_accuracy: 0.8142
Epoch 6/8
609/609 [==============================] - 1252s 2s/step - loss: 0.0525 - accuracy: 0.9755 - val_loss: 0.7364 - val_accuracy: 0.8194
Epoch 7/8
609/609 [==============================] - 1249s 2s/step - loss: 0.0587 - accuracy: 0.9755 - val_loss: 0.7546 - val_accuracy: 0.8319

In [ ]:
metrics=pd.DataFrame(model.history.history)
metrics

,loss,accuracy,val_loss,val_accuracy
0,0.429206,0.812644,0.374831,0.843073
1,0.253533,0.901314,0.397620,0.829941
2,0.112572,0.958621,0.511105,0.828628
3,0.076515,0.971264,0.733899,0.817466
4,0.061203,0.974056,0.661503,0.814183
5,0.052499,0.975534,0.736366,0.819435
6,0.058656,0.975534,0.754569,0.831911
7,0.040994,0.979310,0.849617,0.831911


In [ ]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

In [ ]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)